# Deform a sphere into a cube


---

This notebook solves the problem of deforming a sphere into a cube using ray tracing.


Import the required modules.


In [ ]:
import numpy as np
import pyvista as pv
import utils

Define functions which are used later in the notebook.


---

## 1 - Create source and target meshes


We can create a sphere (source) mesh and a cube (target) mesh using the `utils.sphere()` and `utils.cube()` functions respectively. Generating meshes of spheres and cubes is discussed in the `sphere.ipynb` and `cube.ipynb` notebooks respectively.


In [ ]:
sphere_mesh = utils.sphere(num_polar_angles=20, num_azimuthal_angles=40)
cube_mesh = utils.cube(num_points_per_side=2)

Plot the sphere (source) mesh and the cube (target) mesh.


In [ ]:
pl = pv.Plotter(shape=(1, 2))

pl.subplot(0, 0)
pl.add_mesh(sphere_mesh, show_edges=True)
pl.add_text("Sphere (source)", font_size=12)

pl.subplot(0, 1)
pl.add_mesh(cube_mesh, show_edges=True)
pl.add_text("Cube (target)", font_size=12)

pl.show()

---

## 2 - Deform a sphere into a cube


The first step to solve this problem is to find where the vertex normals of the sphere intersect the faces of the cube. This is a ray tracing problem, and can be solved using PyVista's built in ray tracing functionality. We will define a function, `get_intersection_points()`, which uses PyVista's `ray_trace()` method to find where the sphere's vertex normals intersect the faces of the cube.


In [ ]:
def get_intersection_points(sphere_mesh, cube_mesh, ray_length=10):
    """
    Get intersection points
    =======================

    Calculates where the vertex normals to the sphere mesh intersect the faces of the cube mesh.
    The intersection points are calculated using the PyVista `ray_trace()` method.
    """
    intersection_points = []
    intersection_rays = []
    intersection_cells = []

    # Extract vertices and normals from the sphere mesh.
    sphere_vertices = sphere_mesh.points
    sphere_normals = sphere_mesh.point_data["normals"]

    # Process each ray individually.
    for i, (origin, normal) in enumerate(zip(sphere_vertices, sphere_normals)):
        # Perform ray tracing.
        try:
            end_point = origin + normal * ray_length
            points, cells = cube_mesh.ray_trace(origin, end_point, first_point=True)

            # If intersection found, store the results
            if len(points) > 0:
                intersection_points.append(points)
                intersection_rays.append(i)
                intersection_cells.append(cells)

        except Exception as e:
            continue

    # Convert results to numpy arrays.
    intersection_points = (
        np.array(intersection_points) if intersection_points else np.empty((0, 3))
    )
    intersection_rays = (
        np.array(intersection_rays) if intersection_rays else np.empty((0,), dtype=int)
    )
    intersection_cells = (
        np.array(intersection_cells)
        if intersection_cells
        else np.empty((0,), dtype=int)
    )

    return intersection_points, intersection_rays, intersection_cells

We can use `get_intersection_points()` to solve for the deformation field.


In [ ]:
intersection_points, intersection_rays, intersection_cells = get_intersection_points(
    sphere_mesh,
    cube_mesh,
)

deformation = intersection_points - sphere_mesh.points

print(sphere_mesh.points.shape)
print(intersection_points.shape)

To deform the sphere, we simply need to multiply the deformation field, $\vec{d}$, by a scalar, $t \in [0, 1]$, and evolve the position of each vertex according to this new deformation field. We will define a function, `animate_sphere_to_cube()`, which produces an inline plot of the deformed sphere with an adjustable slider for the deformation parameter, $t$.


In [ ]:
def animate_sphere_to_cube(
    sphere_mesh,
    cube_mesh,
    deformation,
    t_min=0,
    t_max=1,
    show_cube=False,
    show_edges=True,
):
    """
    Animate sphere to cube
    ======================

    Adds an inline plot of the deformed sphere in the notebook, with a slider controlling the deformation parameter.
    """
    sphere_vertices = sphere_mesh.points.copy()

    # Create plotter with custom window size
    pl = pv.Plotter(window_size=[1000, 700])

    # Add reference cube
    if show_cube:
        cube_actor = pl.add_mesh(cube_mesh, color="orange", opacity=0.3)

    # Add initial sphere
    sphere_actor = pl.add_mesh(sphere_mesh, color="lightblue", show_edges=show_edges)

    def update_deformation(t):
        # Calculate deformed vertices
        deformed_vertices = sphere_vertices + deformation * t
        deformed_mesh = pv.PolyData(deformed_vertices, sphere_mesh.faces)

        # Update the mesh
        sphere_actor.GetMapper().SetInputData(deformed_mesh)

        # Render
        pl.render()

    # Add slider
    pl.add_slider_widget(
        update_deformation,
        rng=[t_min, t_max],
        value=0,
        title="t",
        pointa=(0.05, 0.8),
        pointb=(0.25, 0.8),
        style="modern",
    )

    pl.show()

We can now deform the sphere.


In [ ]:
animate_sphere_to_cube(
    sphere_mesh,
    cube_mesh,
    deformation,
)